In [10]:
act_dir = '/checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1636839314/'
def_dir = '/checkpoint/apratik/data/data/apartment_0/default/no_noise/mul_traj_200_combined_2/'

# act_dir = '/checkpoint/apratik/data_devfair0187/apartment_0/straightline/noise/mul_traj_201'
# def_dir = '/checkpoint/apratik/data/data/apartment_0/default/noise/mul_traj_200'

num_traj = 50

import os

def select_eligible(dd, num_traj, pref=''):
    selected = {}
    traj = 1
    while True:
        traj_path = os.path.join(dd, str(traj), pref)
        if not os.path.isdir(traj_path):
            print(f'{traj_path} not found! Skipping ... ')
            traj += 1
            continue
        
        imgs = os.listdir(os.path.join(traj_path, 'rgb'))
#         print(f'{len(imgs)} in {traj_path}')
        if len(imgs) > 100:
            selected[traj] = len(imgs)
        
        traj += 1
        if len(selected.keys()) >= num_traj:
            break
        
    return selected

In [11]:
act_eligible = select_eligible(act_dir, 50, 'default')

In [12]:
def_eligible = select_eligible(def_dir, 50)

In [13]:
for k,v in act_eligible.items():
    print(k, v)

1 1212
2 1709
3 1889
4 1751
5 1616
6 1891
7 1843
8 1507
9 1740
10 1632
11 1829
12 1881
13 1673
14 1833
15 1985
16 1655
17 1459
18 1571
19 1791
20 1410
21 2230
22 1173
23 1978
24 1766
25 1811
26 1831
27 2388
28 1702
29 1811
30 1766
31 1995
32 2152
33 1600
34 1983
35 1847
36 2117
37 2066
38 1489
39 2093
40 1511
41 1257
42 1966
43 2071
44 1997
45 1689
46 2501
47 2105
48 1848
49 1450
50 2116


In [14]:
for k,v in def_eligible.items():
    print(k, v, act_eligible[k])
    if v > act_eligible[k]:
        print('baseline longer')
    else:
        print('active longer')

1 1247 1212
baseline longer
2 2502 1709
baseline longer
3 2335 1889
baseline longer
4 2353 1751
baseline longer
5 2579 1616
baseline longer
6 1915 1891
baseline longer
7 1656 1843
active longer
8 1278 1507
active longer
9 2224 1740
baseline longer
10 2539 1632
baseline longer
11 2991 1829
baseline longer
12 1110 1881
active longer
13 1888 1673
baseline longer
14 2357 1833
baseline longer
15 1343 1985
active longer
16 2744 1655
baseline longer
17 1859 1459
baseline longer
18 2724 1571
baseline longer
19 2339 1791
baseline longer
20 2568 1410
baseline longer
21 2926 2230
baseline longer
22 1672 1173
baseline longer
23 2704 1978
baseline longer
24 2685 1766
baseline longer
25 1334 1811
active longer
26 1304 1831
active longer
27 2590 2388
baseline longer
28 2933 1702
baseline longer
29 2909 1811
baseline longer
30 2707 1766
baseline longer
31 2839 1995
baseline longer
32 2515 2152
baseline longer
33 2938 1600
baseline longer
34 1288 1983
active longer
35 2727 1847
baseline longer
36 2742 

In [15]:
# now create new folder upto the min length
act_out = '/checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1636839314_sampled20_1115_nodownsamplingbaseline'
def_out = '/checkpoint/apratik/data/data/apartment_0/default/no_noise/mul_traj_200_combined_2_sampled20_1115_nodownsamplingbaseline'

print(act_out, def_out)

/checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1636839314_sampled20_1115_nodownsamplingbaseline /checkpoint/apratik/data/data/apartment_0/default/no_noise/mul_traj_200_combined_2_sampled20_1115_nodownsamplingbaseline


In [16]:
def get_traj_length(ae, de, i):
    # get the ith k, v from each dict, return min length
    def get_ith_kv(d, i):
        x = 1
        for k, v in d.items():
            if x == i:
                return k,v
            x += 1
            
    
    ta_i, la_i = get_ith_kv(ae, i)
    td_i, ld_i = get_ith_kv(de, i)
    print(f'{ta_i, la_i, td_i, ld_i}')
    return la_i, ld_i, ta_i, td_i
    
import json
import glob

def acopyjson(a, b, num):
    with open(a, "r") as f:
        dat = json.load(f)
        
    dat2 = {}
    
    for x in range(num):
        dat2.update({x: dat[str(x)]})
        
    with open(b, "w") as fp:
        json.dump(dat2, fp)

from shutil import copyfile

def acopydir(a, b, ext, num):
#     print(f'acopydir {r}')
    if not os.path.isdir(b):
        os.makedirs(b)   
        
    for x in range(num):
        f = "{:05d}{}".format(x, ext)
#         print(f)
        copyfile(os.path.join(a,f), os.path.join(b,f))

def basic_sanity(traj_path):
    def ll(x, y, ext):
        return len(glob.glob(os.path.join(x, y, f'*{ext}')))
    # load json
    with open(os.path.join(traj_path, 'data.json'), 'r') as f:
        data = json.load(f)
    assert ll(traj_path, 'rgb', '.jpg') == ll(traj_path, 'seg', '.npy') == ll(traj_path, 'depth', '.npy') == len(data.keys())
    
def scopy(sd, st, dd, dt, l, pref=''):
    out_dir = os.path.join(dd, str(dt), pref)
    src_dir = os.path.join(sd, str(st), pref)
    
    print(f'sampling first {l} things from src_dir {src_dir}, out_dir {out_dir}')
    assert dd != sd
    
    if not os.path.isdir(out_dir):
        print(f'creating {out_dir}')
        os.makedirs(out_dir)
    
    acopydir(os.path.join(src_dir, 'rgb'), os.path.join(out_dir, 'rgb'), '.jpg', l)
    acopydir(os.path.join(src_dir, 'depth'), os.path.join(out_dir, 'depth'), '.npy', l)
    acopydir(os.path.join(src_dir, 'seg'), os.path.join(out_dir, 'seg'), '.npy', l)
    acopyjson(os.path.join(src_dir, 'data.json'), os.path.join(out_dir, 'data.json'), l)
    if pref == 'default':
        copyfile(os.path.join(src_dir, 'active_frames.txt'), os.path.join(out_dir, 'active_frames.txt'))
    
    basic_sanity(out_dir)
    
for i in range(20, 51):
    la, ld, act_traj, def_traj = get_traj_length(act_eligible, def_eligible, i)
    print(f'{la, ld, act_traj, def_traj}')
    
    # now copy minl things 
    scopy(act_dir, act_traj, act_out, i, min(la, ld), 'default')
#     scopy(def_dir, def_traj, def_out, i, ld)
    

(20, 1410, 20, 2568)
(1410, 2568, 20, 20)
sampling first 1410 things from src_dir /checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1636839314/20/default, out_dir /checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1636839314_sampled20_1115_nodownsamplingbaseline/20/default
(21, 2230, 21, 2926)
(2230, 2926, 21, 21)
sampling first 2230 things from src_dir /checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1636839314/21/default, out_dir /checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1636839314_sampled20_1115_nodownsamplingbaseline/21/default
creating /checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1636839314_sampled20_1115_nodownsamplingbaseline/21/default
(22, 1173, 22, 1672)
(1173, 1672, 22, 22)
sampling first 1173 things from src_dir /checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1636839314/22/default, out_dir /checkpoint/apratik/data_devfair0187/apartment